## Importanto pacotes e arquivos

In [132]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
#np.set_printoptions(threshold=sys.maxsize)

In [2]:
foto = "imagens/1950/LMH-OMV-1949-1953-52-53 (H) - 2.jpg"
adress = "imagens/1950"

## Pré-processamento (cortando, transformação de perspectiva, RGB2GRAY, blur gaussiano e normalização de contraste por blocos)

In [124]:
img = cv2.imread(foto)
coords = []
selected_index = -1

def drawQuadrilateral(event, x, y, flags, param):
    global coords, selected_index

    if event == cv2.EVENT_LBUTTONDOWN:
        for i, (px, py) in enumerate(coords):
            if np.abs(x - px) < 10 and np.abs(y - py) < 10:
                selected_index = i
                break
        else:
            coords.append((x, y))
            selected_index = len(coords) - 1

    elif event == cv2.EVENT_LBUTTONUP:
        selected_index = -1

    elif event == cv2.EVENT_MOUSEMOVE and selected_index != -1:
        coords[selected_index] = (x, y)

    img_copy = img.copy()

    for i, (px, py) in enumerate(coords):
        cv2.circle(img_copy, (px, py), 3, (0, 0, 255), -1)
        cv2.putText(img_copy, f"{i + 1}", (px - 10, py - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    if len(coords) == 4:
        pts = np.array(coords, np.int32)
        cv2.polylines(img_copy, [pts], True, (255, 0, 0), 2)

    cv2.imshow("Ajuste do Quadrilátero", img_copy)


cv2.namedWindow("Ajuste do Quadrilátero")
cv2.setMouseCallback("Ajuste do Quadrilátero", drawQuadrilateral)

cv2.imshow("Ajuste do Quadrilátero", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

pts1 = np.float32([coords[0], coords[1], coords[3], coords[2]])


pts2 = np.float32([[0, 0], [1890, 0], [0, 1080], [1890, 1080]])

M = cv2.getPerspectiveTransform(pts1, pts2)

img_dist = cv2.warpPerspective(img, M, (1896, 200))

img = cv2.cvtColor(img_dist, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=8.5, tileGridSize=(136,36))
img = clahe.apply(img)
img = cv2.GaussianBlur(img, (1, 1),16)
#img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,45,50) 
imgTratada = img.copy()
cv2.imwrite(adress+'/imagem tratada.jpg', imgTratada)

rows, cols = 5, 24 #(pode ser um input do usuário, sempre mantendo o número de linhas
                    #da tabela n menos 1: rowls = n-1

cell_size = img.shape[0] // rows, img.shape[1] // cols
# pegamos o tamanho da imagem com img.shape e dividimos pelo número de linhas e colunas 
# desejadas para ter o tamanho de cada célula

for i in range(rows):
    y = i * cell_size[0]
    cv2.line(img, (0, y), (img.shape[1], y), (0, 255, 0), 1)

for j in range(cols):
    x = j * cell_size[1]
    cv2.line(img, (x, 0), (x, img.shape[0]), (0, 255, 0), 1)

cv2.imshow("Imagem com grid", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(adress+'/imagem sem distorcao.png',img_dist)


True

## Dividindo a imagem em células da tabela

In [125]:
img = cv2.imread(adress+"/imagem tratada.jpg")
cells1 = []
cell_size = img.shape[0] // rows, img.shape[1] // cols

for i in range(rows):
    for j in range(cols):
        y1, y2 = i * cell_size[0], (i + 1) * cell_size[0]
        x1, x2 = j * cell_size[1], (j + 1) * cell_size[1]
        #criando labels coordenadas para cada célula
        cell = img[y1:y2, x1:x2].copy() 
        cells1.append(cell) #adiciona a célula na lista
        #cellsArray = np.array(cells)

In [126]:
np.save(adress+'/cells-1-1',cells1)

## Formatando as imagens e dividindo as células em dígitos

In [158]:
def resizingImages(array):
    
    largura = 18
    altura = 36
    target_resized = []
    
    for i in range(len(array)):
        
        img = array[i]
        img = img.reshape(-1,36,18,1)
        target_resized.append(img)
        
        #target_resized.append(cv2.resize(img, (largura, altura)))
        
    array = np.array(target_resized)    
    return array

In [238]:
digits1 = []
largura = 18
altura = 36
for i in range(len(cells1)):
    teste = cv2.cvtColor(cells1[i], cv2.COLOR_BGR2GRAY)
    digits1.append(cv2.resize(teste[:,25:37], (largura, altura)))
    digits1.append(cv2.resize(teste[:,36:50], (largura, altura)))
    digits1.append(cv2.resize(teste[:,49:63], (largura, altura)))
digits1 = np.array(digits1)
digits1 = digits1.reshape(-1,36,18,1)


In [240]:
np.save(adress+'/digits-1-6',digits1)